# Plots of LO frequency drift transforms
Refer to:


In [ ]:
import torchsig.transforms.functional as F

import numpy as np
import scipy as sp
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# test functional
rng = np.random.default_rng(42)

N = 10000
center_frequency = 0
sample_rate = 10e6
n = np.arange(0,N)
t = n/sample_rate
tone_bb_data = np.exp(2j*np.pi*center_frequency*t)

drift_std=100
data_out = F.local_oscillator_frequency_drift(
    data = tone_bb_data,
    drift_std = drift_std,
    sample_rate=sample_rate,
    rng = rng
)

In [ ]:
fig = plt.figure(figsize=(10,6))
fig.subplots_adjust(hspace=0.5)
ax = fig.add_subplot(2,1,1)
ax.plot(t,np.real(tone_bb_data),label='Real, Input Tone')
ax.plot(t,np.imag(tone_bb_data),label='Imag, Input Tone')
ylim = np.max(np.abs(tone_bb_data))*1.1
ax.set_ylim([-ylim,ylim])
ax.set_xlim([t[0],t[-1]])
ax.set_xlabel('Time (s)')
ax.set_ylabel('Amplitude')
ax.grid()
ax.legend(loc='upper right')

ax = fig.add_subplot(2,1,2)
ax.plot(t,np.real(data_out),label='Real, Tone with LO Drift')
ax.plot(t,np.imag(data_out),label='Imag, Tone with LO Drift')
ax.set_ylim([-ylim,ylim])
ax.set_xlim([t[0],t[-1]])
ax.set_xlabel('Time (s)')
ax.set_ylabel('Amplitude')
ax.grid()
ax.legend(loc='upper right')

In [ ]:
win = sp.signal.windows.blackmanharris(len(tone_bb_data))
fft_size = 2**20
f = np.linspace(-0.5,0.5-(1/fft_size),fft_size)*sample_rate
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(1,1,1)
ax.plot(f,20*np.log10(np.abs(np.fft.fftshift(np.fft.fft(tone_bb_data*win,fft_size)))),label='Input Tone')
ax.plot(f,20*np.log10(np.abs(np.fft.fftshift(np.fft.fft(data_out*win,fft_size)))),'--',label='Output Tone with LO Drift')
ax.legend(loc='upper right')
ax.set_xlim([-100000+center_frequency,100000+center_frequency])
ax.grid()
ax.set_ylabel('Magnitude (dB)')
ax.set_xlabel('Frequency (Hz)')


In [ ]:
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(1,1,1)
unwrapped_phase = np.unwrap(np.angle(data_out))*(sample_rate/(2*np.pi))
instantaneous_frequency = np.diff(unwrapped_phase)
ax.plot(t[0:1000],instantaneous_frequency[0:1000])
ax.plot(t[0:1000],drift_std*np.ones(1000),'r',label='Drift Standard Deviation')
ax.plot(t[0:1000],-drift_std*np.ones(1000),'r')
ax.set_title('Instantaneous Frequency of Signal with LO Drift')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Frequency (Hz)')
ax.grid()
ax.legend(loc='upper right')
